In [1]:
import tensorflow.keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, BatchNormalization, LocallyConnected2D, SeparableConv2D
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.constraints import max_norm
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import activations

In [2]:
from import_data import import_data

In [3]:
X, y, _ = import_data()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(y_train.shape)

153 parties rejetées par le goban, reste 5835 parties
(37344, 9, 9, 4)
(37344, 82)


In [18]:
model = Sequential()
model.add(Conv2D(64, kernel_size=3, input_shape=X[0].shape, padding ="same"))
model.add(Conv2D(64, kernel_size=3, padding ="same"))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=3, padding ="same"))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(82, activation='sigmoid')) 

opt = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mae', optimizer=opt)
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 9, 9, 64)          2368      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_10 (Batc (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_11 (Batc (None, 9, 9, 64)          256       
_________________________________________________________________
flatten_5 (Flatten)          (None, 5184)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)              

In [19]:
# hyperparamètres
epochs = 30
batch_size = 64

# training
print("Training ...")
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))

Training ...
Epoch 1/30
584/584 [==============================] - 4s 7ms/step - loss: 0.1440 - val_loss: 0.1263
Epoch 2/30
584/584 [==============================] - 4s 7ms/step - loss: 0.1262 - val_loss: 0.1164
Epoch 3/30
584/584 [==============================] - 4s 7ms/step - loss: 0.1177 - val_loss: 0.1060
Epoch 4/30
584/584 [==============================] - 4s 7ms/step - loss: 0.1074 - val_loss: 0.0886
Epoch 5/30
584/584 [==============================] - 4s 7ms/step - loss: 0.0961 - val_loss: 0.0775
Epoch 6/30
584/584 [==============================] - 4s 7ms/step - loss: 0.0902 - val_loss: 0.0714
Epoch 7/30
584/584 [==============================] - 4s 7ms/step - loss: 0.0866 - val_loss: 0.0678
Epoch 8/30
584/584 [==============================] - 4s 7ms/step - loss: 0.0839 - val_loss: 0.0637
Epoch 9/30
584/584 [==============================] - 4s 7ms/step - loss: 0.0821 - val_loss: 0.0613
Epoch 10/30
584/584 [==============================] - 4s 7ms/step - loss: 0.0805 - val

In [15]:
proba = model.predict(X_train)
print(proba[5])
print(sum(proba[5]))

[2.5617101e-03 1.2980124e-03 2.6070706e-03 1.9202434e-03 7.9850294e-03
 9.7885007e-01 4.5369985e-04 4.9124775e-04 9.8596979e-04 7.7178515e-04
 4.2480476e-02 2.9640496e-01 9.6044067e-04 1.9379430e-03 3.6592403e-04
 7.0451538e-04 1.2362213e-03 5.3607812e-04 6.6860928e-03 9.7493705e-04
 2.0564075e-03 9.5689367e-04 1.2840690e-03 5.4458168e-04 5.0091615e-04
 2.2010817e-03 1.5298569e-03 3.9029205e-03 2.2861476e-03 1.0539457e-03
 1.8680214e-04 3.1058947e-04 3.8917828e-04 1.4353293e-04 9.5648889e-04
 1.1224241e-03 4.3201428e-03 4.4759154e-02 1.8706948e-03 6.1538506e-02
 2.5476972e-04 2.0249073e-04 5.7889154e-04 9.2975469e-04 1.0931070e-03
 9.8309213e-01 1.8251343e-01 1.8199503e-06 2.7075550e-05 3.1672241e-04
 1.4558128e-03 2.3876356e-04 2.7031958e-04 1.0871741e-03 1.6826808e-02
 2.6248729e-01 2.0463082e-01 3.4840909e-01 4.1328100e-04 3.0808622e-04
 4.7657406e-04 1.0665577e-03 6.0201291e-04 4.2949259e-02 6.7785704e-01
 9.5647031e-01 9.9844090e-04 7.4478769e-04 5.5926939e-04 6.9458445e-04
 1.936